# MA4702 Laboratorio 2 2022
## Generación de Columnas: Problema sobre Emparejamientos Perfectos
## Integrantes del grupo
- Alonso Rojas
- Cristóbal Ramos
- Karim Saud
- Manuel Torres

## Observaciones
Si tiene alguna observación a la corrección escríbala acá (claridad del enunciado, díficultades con Julia u otros).

# 1. Preparación

Usaremos los paquetes JuMP, Gurobi. Descomente los siguientes comandos si no tiene los paquetes y asegúrese de tener la última versión de JuMP corriendo. Vuelvalos a comentar una vez que los ejecute.

In [1]:
import Pkg
Pkg.update("JuMP")
Pkg.status("JuMP")

      Status `C:\Users\Personal\.julia\environments\v1.7\Project.toml`
  [4076af6c] JuMP v1.0.0


    Updating registry at `C:\Users\Personal\.julia\registries\General.toml`
   Installed RecipesPipeline ── v0.5.2
   Installed StatsAPI ───────── v1.2.2
   Installed GR_jll ─────────── v0.64.2+0
   Installed Compat ─────────── v3.43.0
   Installed Expat_jll ──────── v2.4.8+0
   Installed LogExpFunctions ── v0.3.12
   Installed Parsers ────────── v2.2.4
   Installed Latexify ───────── v0.15.14
   Installed StaticArrays ───── v1.4.3
   Installed JuMP ───────────── v1.0.0
   Installed MathOptInterface ─ v1.1.2
   Installed GR ─────────────── v0.64.2
    Updating `C:\Users\Personal\.julia\environments\v1.7\Project.toml`
  [4076af6c] ↑ JuMP v0.23.2 ⇒ v1.0.0
    Updating `C:\Users\Personal\.julia\environments\v1.7\Manifest.toml`
  [34da2185] ↑ Compat v3.42.0 ⇒ v3.43.0
  [28b8d3ca] ↑ GR v0.64.0 ⇒ v0.64.2
  [4076af6c] ↑ JuMP v0.23.2 ⇒ v1.0.0
  [23fbe1c1] ↑ Latexify v0.15.13 ⇒ v0.15.14
  [2ab3a3ac] ↑ LogExpFunctions v0.3.10 ⇒ v0.3.12
  [b8f27783] ↑ MathOptInterface v1.1.0 ⇒ v1.1.2
  [69de0a69]

In [2]:
using JuMP, Gurobi
# Abrir un solo ambiente de Gurobi, Jupyter se marea si se crean demasiados modelos durante en una misma celda.
const GUROBI_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-09


┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1423
┌ Warning: Module JSON with build ID 20981288423301 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1107
┌ Warning: Module JSON with build ID 20981288423301 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1107
┌ Warning: Module JSON with build ID 20981288423301 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1107
┌ Warning: Module JSON with build ID 20981288423301 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that d

Gurobi.Env(Ptr{Nothing} @0x0000000051ddb650, false, 0)

# 2. Funciones Auxiliares

Pueden obtener las instancias de prueba para sus programas gracias a las siguientes funciones:

In [55]:
# Crea las instancias de prueba de la función resolverProblemaMaestro.
function crearInstancias(j)
    A = [[1 1 1 0 1 0 -1 0 1 -1; 0 -1 0 0 -1 -1 0 0 1 0; -1 1 0 -1 1 -1 1 1 -1 -1; 0 1 0 0 1 1 0 -1 1 1; -1 0 -1 1 -1 0 1 -1 -1 1], [1 0 0 -1 0 -1 -1 0 1 -1 1 1; -1 -1 0 1 1 0 1 -1 0 1 1 0; 0 1 0 1 -1 1 0 0 0 -1 -1 -1; 1 -1 1 1 -1 -1 1 1 1 1 -1 1; 1 -1 0 1 -1 -1 0 -1 1 -1 1 1; 1 1 1 -1 0 1 -1 1 1 -1 0 -1], [1 1 1 0 0 1 1 -1 0 0 0 -1 -1 0; 0 0 0 1 -1 1 0 1 0 0 -1 1 1 -1; 0 -1 0 -1 1 1 0 -1 0 0 -1 1 0 -1; 1 1 1 0 -1 0 0 -1 0 -1 0 -1 -1 -1; -1 0 0 1 1 0 -1 0 -1 1 0 1 1 0; 0 1 -1 1 1 -1 0 1 -1 0 0 0 0 -1; 0 0 0 1 0 1 1 -1 0 -1 0 0 0 1], [-1 1 -1 0 0 1 0 -1 0 0 -1 1 -1 1 -1 -1; -1 -1 1 0 -1 -1 0 0 0 1 1 -1 -1 -1 0 -1; -1 0 -1 1 0 0 0 -1 1 -1 -1 1 -1 -1 0 1; -1 -1 0 1 1 1 -1 -1 -1 -1 0 1 -1 -1 1 1; 1 0 -1 0 0 1 -1 0 0 1 -1 0 1 0 1 -1; 0 0 -1 0 0 -1 0 -1 0 1 0 0 -1 0 -1 1; -1 0 0 0 0 0 1 0 0 1 -1 0 -1 0 -1 1; 0 1 -1 1 0 1 0 0 1 -1 -1 -1 1 1 -1 -1], [-1 -1 0 -1 -1 0 1 1 1 -1 -1 1 1 -1 1 -1 -1 -1; 1 -1 0 0 -1 0 1 1 -1 -1 -1 0 0 -1 0 1 -1 -1; 1 -1 0 1 -1 1 -1 0 0 1 -1 -1 -1 1 1 0 0 0; 1 -1 -1 -1 -1 0 -1 -1 0 1 -1 0 0 1 -1 1 0 -1; -1 0 1 -1 -1 -1 -1 1 0 1 1 0 0 1 1 -1 0 -1; 0 -1 0 -1 0 1 0 -1 1 0 -1 -1 0 1 0 1 1 0; -1 0 1 -1 0 -1 0 -1 -1 0 -1 0 0 1 1 -1 -1 0; 0 0 1 -1 1 1 -1 -1 -1 -1 -1 0 1 -1 -1 -1 0 0; 1 -1 1 0 0 0 0 -1 -1 -1 -1 1 -1 1 1 0 1 1], [0 1 0 0 1 1 -1 1 0 0 1 0 0 -1 0 -1 1 0 0 -1; -1 1 1 1 0 0 1 1 0 0 0 1 0 -1 -1 -1 1 0 1 0; 1 -1 0 -1 1 1 -1 1 0 0 1 1 -1 0 -1 1 1 0 -1 1; 1 -1 0 1 -1 -1 1 0 1 1 -1 -1 1 -1 -1 0 0 1 1 -1; -1 1 -1 -1 -1 1 -1 -1 1 -1 1 1 0 -1 -1 -1 -1 0 0 0; 0 0 0 0 1 1 -1 1 1 1 1 -1 1 -1 1 -1 1 0 -1 -1; 1 -1 0 0 -1 -1 1 1 0 0 -1 0 1 1 -1 1 0 0 0 1; 0 0 -1 -1 1 0 -1 1 0 0 0 -1 1 0 0 1 -1 1 1 0; -1 -1 1 -1 1 -1 0 -1 1 0 -1 0 0 -1 0 -1 1 -1 1 1; 0 1 0 1 0 0 0 1 -1 1 -1 -1 1 -1 1 0 -1 -1 -1 -1], [0 -1 1 1 1 0 -1 -1 1 1 -1 1 0 0 1 0 1 0 -1 -1 0 0; -1 -1 -1 -1 -1 -1 -1 -1 1 -1 0 0 1 1 0 0 0 -1 1 0 -1 -1; 0 -1 0 0 0 0 1 0 -1 0 0 1 0 -1 1 1 -1 0 0 1 0 -1; 0 1 -1 1 1 -1 -1 1 0 1 1 -1 0 -1 1 -1 1 1 -1 1 1 1; 1 1 1 -1 -1 -1 -1 1 1 -1 -1 1 -1 1 1 1 0 0 -1 1 0 1; 1 -1 0 1 -1 -1 1 1 0 -1 0 0 0 -1 1 1 -1 -1 0 0 0 -1; 0 0 1 1 0 1 1 -1 -1 -1 1 -1 0 0 -1 -1 -1 1 0 0 1 0; 1 1 0 -1 1 -1 1 1 0 1 0 -1 -1 -1 1 0 -1 -1 0 -1 1 1; -1 1 -1 0 -1 -1 1 1 0 1 0 -1 0 1 1 -1 -1 0 -1 0 1 0; -1 0 0 -1 -1 0 -1 0 1 0 0 0 -1 -1 -1 0 -1 1 0 1 0 -1; -1 0 0 -1 1 1 -1 1 -1 1 0 -1 0 0 1 -1 1 0 0 0 1 -1], [0 0 0 0 1 1 1 -1 1 0 1 1 1 -1 -1 -1 1 1 1 -1 1 1 -1 0; -1 1 1 1 -1 0 1 1 1 1 1 -1 0 -1 0 1 -1 -1 -1 -1 -1 1 0 -1; -1 -1 0 1 1 1 -1 0 0 1 -1 1 -1 1 0 -1 -1 0 0 -1 0 -1 1 0; 0 -1 -1 0 0 1 0 0 1 -1 0 -1 -1 -1 0 1 -1 0 -1 0 -1 -1 0 -1; 1 1 0 0 -1 0 0 -1 0 0 -1 1 0 0 0 1 1 -1 -1 1 -1 0 -1 1; 0 1 1 0 0 1 1 -1 -1 1 -1 1 1 -1 0 1 1 -1 -1 -1 0 -1 0 1; -1 -1 0 0 -1 -1 -1 1 0 0 -1 -1 1 -1 0 -1 0 0 -1 -1 -1 1 0 -1; 1 -1 -1 1 1 1 -1 -1 0 -1 -1 -1 0 -1 -1 0 0 -1 -1 1 -1 0 -1 0; -1 1 0 -1 -1 1 -1 -1 1 0 -1 -1 0 1 -1 -1 0 0 1 0 1 1 -1 0; 0 -1 0 -1 1 -1 1 1 -1 0 1 -1 1 1 1 0 0 1 -1 -1 1 1 1 -1; 1 1 1 -1 -1 -1 -1 -1 0 1 -1 -1 1 0 -1 -1 1 0 0 1 1 0 0 0; 1 0 0 0 -1 -1 0 1 0 0 0 -1 -1 -1 -1 -1 1 1 1 1 -1 1 0 0], [0 -1 -1 1 1 0 0 0 -1 1 1 1 -1 -1 -1 -1 1 0 1 0 -1 0 0 1 -1 1; 0 0 1 -1 0 0 1 -1 1 0 1 -1 -1 1 0 -1 1 0 0 -1 0 -1 0 1 -1 -1; -1 -1 -1 -1 0 1 0 -1 0 0 1 -1 0 0 1 -1 0 0 -1 1 -1 0 0 -1 -1 1; 0 -1 -1 -1 -1 1 1 1 1 0 1 0 1 1 1 1 0 0 -1 -1 -1 1 0 0 1 0; 0 0 -1 0 -1 0 0 0 0 -1 1 1 -1 0 1 -1 1 -1 0 0 -1 -1 0 1 1 1; 1 -1 0 -1 0 -1 0 -1 1 0 0 1 -1 -1 0 -1 1 1 0 1 -1 1 0 0 -1 0; 0 -1 0 -1 1 0 0 1 1 0 -1 0 -1 0 0 0 0 -1 -1 1 -1 1 -1 -1 0 -1; 1 -1 -1 1 -1 0 1 0 0 -1 -1 -1 1 1 -1 0 0 -1 -1 1 0 1 0 1 -1 1; 0 0 -1 0 1 0 0 1 1 1 1 0 1 1 0 0 -1 -1 1 -1 0 -1 0 -1 1 1; -1 0 -1 -1 1 0 -1 1 1 -1 1 0 -1 0 -1 0 0 1 1 -1 1 -1 -1 1 -1 -1; 1 0 -1 0 0 0 -1 -1 -1 0 1 1 -1 -1 0 1 -1 -1 -1 0 0 1 1 0 1 1; -1 -1 0 0 1 1 1 0 1 1 0 0 -1 -1 0 0 -1 0 -1 1 0 0 -1 0 -1 1; 0 0 1 -1 1 1 -1 0 -1 1 1 -1 -1 1 0 0 1 -1 0 0 1 0 0 -1 -1 0], [-1 0 0 1 1 0 1 -1 0 1 -1 0 -1 -1 0 -1 1 1 0 0 -1 1 -1 -1 0 0 0 -1; 1 0 1 -1 1 1 1 -1 0 -1 1 1 1 0 0 -1 1 -1 1 1 1 1 0 0 0 0 -1 0; -1 1 0 1 0 0 -1 0 -1 1 0 1 1 -1 -1 -1 0 -1 1 1 -1 -1 0 1 1 -1 0 -1; -1 1 0 -1 -1 1 1 -1 -1 -1 0 1 1 -1 -1 0 -1 1 1 0 1 0 -1 -1 1 1 1 0; -1 1 -1 1 1 -1 -1 -1 1 1 0 -1 1 0 1 0 0 1 -1 0 -1 1 0 -1 1 0 0 1; 0 1 -1 0 0 0 -1 0 -1 0 -1 0 0 0 1 0 0 -1 -1 0 1 0 1 0 -1 1 -1 -1; 1 1 -1 1 1 -1 -1 0 -1 1 -1 -1 0 1 0 1 -1 1 -1 0 -1 -1 1 -1 1 1 -1 -1; 0 1 1 -1 1 0 0 -1 -1 1 0 0 0 0 -1 0 1 -1 1 0 1 -1 1 0 -1 1 0 -1; 0 -1 1 1 -1 0 1 -1 0 0 1 0 -1 1 -1 1 0 -1 1 -1 0 0 0 0 -1 0 -1 1; -1 0 0 0 -1 -1 -1 0 0 1 1 1 -1 0 -1 1 1 -1 -1 1 -1 -1 1 1 1 -1 1 0; 0 0 1 0 1 -1 -1 0 0 1 1 0 1 -1 0 1 -1 0 1 0 -1 -1 -1 -1 1 0 1 -1; 1 1 -1 0 1 -1 -1 -1 0 0 -1 0 1 -1 -1 -1 0 0 0 -1 1 -1 1 0 0 0 -1 1; -1 1 1 0 1 0 -1 0 1 -1 0 -1 1 0 0 1 0 1 -1 -1 1 1 0 0 0 1 -1 1; -1 0 -1 -1 1 1 1 0 1 0 1 -1 0 0 1 1 0 1 1 1 0 -1 1 -1 1 1 -1 -1], [1 1 0 1 0 1 -1 1 0 1 1 1 -1 0 0 0 0 1 0 -1 -1 0 1 -1 -1 0 -1 0 1 -1; -1 0 0 0 0 -1 -1 -1 -1 0 1 1 0 0 -1 0 1 -1 0 0 0 0 -1 1 0 0 -1 -1 1 1; 1 -1 1 0 0 1 -1 -1 0 1 1 -1 -1 1 0 1 0 0 0 0 -1 -1 1 1 0 0 0 0 -1 1; 1 0 0 -1 1 1 1 -1 0 -1 1 0 0 1 -1 0 1 0 1 -1 1 1 0 0 1 1 0 1 1 0; 1 0 1 -1 1 0 0 -1 0 -1 -1 0 0 -1 1 1 0 1 -1 1 0 1 -1 -1 1 1 0 1 1 1; -1 1 0 -1 1 -1 0 -1 -1 -1 -1 1 1 -1 0 -1 -1 1 0 -1 -1 1 1 0 1 0 0 0 -1 0; -1 0 -1 1 -1 0 0 1 0 1 1 1 1 1 0 -1 0 -1 0 0 -1 1 0 0 1 1 1 -1 -1 -1; -1 0 0 0 -1 -1 1 0 0 -1 0 -1 1 1 -1 1 1 1 0 -1 0 1 1 0 1 0 -1 1 0 -1; 1 1 0 -1 0 1 0 1 -1 0 -1 1 0 0 0 -1 1 1 1 1 1 1 0 0 1 0 0 1 1 1; -1 1 -1 -1 0 0 0 0 0 1 0 -1 -1 0 -1 0 0 -1 0 -1 1 1 0 0 -1 1 1 -1 -1 0; 1 0 1 1 -1 -1 0 -1 1 0 1 1 1 1 0 0 0 -1 1 1 0 1 1 1 -1 -1 -1 -1 0 1; 1 -1 -1 1 -1 1 -1 0 -1 0 0 0 -1 1 1 -1 0 1 -1 1 0 0 0 -1 0 1 0 1 0 0; 1 1 0 -1 0 1 0 0 -1 1 1 0 1 0 1 0 -1 0 0 -1 -1 1 0 1 -1 -1 1 0 1 -1; 1 0 1 -1 -1 -1 1 0 1 0 0 -1 -1 0 0 1 1 1 -1 -1 1 0 1 0 0 0 1 0 1 -1; 1 0 0 -1 0 1 0 -1 0 -1 0 -1 1 0 -1 0 1 0 -1 -1 0 -1 0 -1 -1 0 1 -1 0 0]]
    g = [[-0.2, 0.0, 0.2, 0.2, 0.2], [0.5, 0.3333333333333333, -0.3333333333333333, 0.0, 0.6666666666666666, 0.16666666666666666], [0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, -0.2857142857142857, -0.2857142857142857, 0.0], [-0.5, -0.5, 0.0, -0.125, 0.125, -0.5, -0.5, -0.25], [0.1111111111111111, 0.3333333333333333, -0.4444444444444444, -0.2222222222222222, -0.2222222222222222, -0.7777777777777778, 0.0, 0.3333333333333333, -0.1111111111111111], [0.0, 0.0, 0.2, -0.2, 0.3, 0.0, 0.4, -0.3, 0.2, 0.1], [-0.09090909090909091, 0.0, -0.09090909090909091, -0.09090909090909091, 0.09090909090909091, 0.0, -0.18181818181818182, 0.5454545454545454, -0.2727272727272727, -0.2727272727272727, 0.36363636363636365], [0.0, 0.08333333333333333, -0.16666666666666666, -0.08333333333333333, 0.16666666666666666, -0.3333333333333333, 0.16666666666666666, -0.3333333333333333, 0.16666666666666666, -0.3333333333333333, -0.16666666666666666, -0.16666666666666666], [0.0, -0.46153846153846156, 0.23076923076923078, 0.07692307692307693, 0.3076923076923077, 0.5384615384615384, -0.46153846153846156, 0.0, 0.15384615384615385, -0.23076923076923078, -0.3076923076923077, -0.15384615384615385, -0.3076923076923077], [-0.2857142857142857, 0.0, 0.42857142857142855, -0.21428571428571427, -0.14285714285714285, 0.07142857142857142, 0.2857142857142857, -0.21428571428571427, 0.07142857142857142, 0.14285714285714285, 0.0, 0.21428571428571427, 0.2857142857142857, -0.2857142857142857], [0.3333333333333333, 0.06666666666666667, -0.13333333333333333, -0.06666666666666667, -0.2, 0.26666666666666666, 0.06666666666666667, -0.13333333333333333, 0.26666666666666666, -0.06666666666666667, -0.06666666666666667, 0.26666666666666666, 0.06666666666666667, 0.06666666666666667, 0.2]]
    d = [[0.4, 0.0, -0.2, -0.2, 0.0], [-0.16666666666666666, 0.3333333333333333, -0.16666666666666666, 0.0, -0.3333333333333333, 0.16666666666666666], [0.2857142857142857, 0.42857142857142855, 0.42857142857142855, -0.2857142857142857, -0.14285714285714285, 0.14285714285714285, -0.42857142857142855], [0.0, 0.375, 0.25, -0.5, 0.125, 0.125, -0.125, 0.0], [-0.6666666666666666, -0.5555555555555556, 0.1111111111111111, -0.2222222222222222, -0.1111111111111111, 0.0, 0.3333333333333333, -0.1111111111111111, -0.3333333333333333], [0.0, -0.1, 0.1, 0.5, 0.2, -0.1, -0.6, -0.2, 0.0, -0.1], [0.0, -0.09090909090909091, -0.36363636363636365, 0.2727272727272727, 0.09090909090909091, -0.18181818181818182, -0.18181818181818182, -0.09090909090909091, 0.0, 0.18181818181818182, -0.18181818181818182], [0.0, 0.0, 0.16666666666666666, -0.16666666666666666, 0.16666666666666666, -0.3333333333333333, -0.3333333333333333, -0.16666666666666666, -0.08333333333333333, -0.16666666666666666, 0.25, -0.16666666666666666], [-0.5384615384615384, -0.38461538461538464, 0.23076923076923078, 0.0, 0.07692307692307693, 0.0, 0.07692307692307693, -0.23076923076923078, -0.3076923076923077, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.3076923076923077], [0.5, 0.07142857142857142, -0.14285714285714285, -0.5, 0.2857142857142857, 0.0, -0.21428571428571427, 0.14285714285714285, 0.0, 0.07142857142857142, -0.21428571428571427, -0.2857142857142857, 0.2857142857142857, -0.21428571428571427], [0.2, -0.26666666666666666, 0.13333333333333333, -0.26666666666666666, 0.0, 0.06666666666666667, 0.26666666666666666, 0.0, 0.2, -0.26666666666666666, 0.4666666666666667, 0.0, 0.2, 0.0, 0.0]]
    return A[j], g[j], d[j]
end

# Retorna la cantidad de vertices V, la cantidad de aristas del grafo bipartito m, y sus aristas E.
function crearGrafoBipartito()
    V = 20
    m = 194
    E = [[1, 2],[1, 4],[1, 6],[1, 8],[1, 10],[1, 12],[1, 14],[1, 16],[1, 18],[1, 20],[2, 3],[2, 5],[2, 7],[2, 9],[2, 11],[2, 13],[2, 15],[2, 17],[2, 19],[3, 4],[3, 6],[3, 8],[3, 10],[3, 12],[3, 14],[3, 16],[3, 18],[3, 20],[4, 5],[4, 7],[4, 9],[4, 11],[4, 13],[4, 15],[4, 17],[4, 19],[5, 6],[5, 8],[5, 10],[5, 12],[5, 14],[5, 16],[5, 18],[5, 20],[6, 7],[6, 9],[6, 11],[6, 13],[6, 15],[6, 17],[6, 19],[7, 8],[7, 10],[7, 12],[7, 16],[7, 18],[7, 20],[8, 9],[8, 11],[8, 13],[8, 15],[8, 17],[8, 19],[9, 10],[9, 14],[9, 16],[9, 18],[9, 20],[10, 11],[10, 13],[10, 17],[10, 19],[11, 12],[11, 14],[11, 16],[11, 18],[11, 20],[12, 13],[12, 15],[12, 17],[12, 19],[13, 14],[13, 16],[13, 18],[13, 20],[14, 15],[14, 17],[14, 19],[15, 16],[15, 18],[15, 20],[16, 17],[16, 19],[17, 18],[17, 20],[18, 19],[19, 20]]
    return V, m, E
end

# Crea una instancia de prueba para resolverProblemaPricing.
function crearTestY()
    y = [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1; 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1; 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
    return y
end

# Crea una instancia de prueba para resolverGeneracionColumnas.
function crearParametros()
    g = [0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19; 1 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18; 2 1 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17; 3 2 1 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16; 4 3 2 1 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15; 5 4 3 2 1 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14; 6 5 4 3 2 1 0 1 2 3 4 5 6 7 8 9 10 11 12 13; 7 6 5 4 3 2 1 0 1 2 3 4 5 6 7 8 9 10 11 12; 8 7 6 5 4 3 2 1 0 1 2 3 4 5 6 7 8 9 10 11; 9 8 7 6 5 4 3 2 1 0 1 2 3 4 5 6 7 8 9 10; 10 9 8 7 6 5 4 3 2 1 0 1 2 3 4 5 6 7 8 9; 11 10 9 8 7 6 5 4 3 2 1 0 1 2 3 4 5 6 7 8; 12 11 10 9 8 7 6 5 4 3 2 1 0 1 2 3 4 5 6 7; 13 12 11 10 9 8 7 6 5 4 3 2 1 0 1 2 3 4 5 6; 14 13 12 11 10 9 8 7 6 5 4 3 2 1 0 1 2 3 4 5; 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 0 1 2 3 4; 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 0 1 2 3; 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 0 1 2; 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 0 1; 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 0]
    d = [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5; 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5; 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5; 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5; 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5; 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5; 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5; 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5; 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5; 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5; 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5; 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5; 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5; 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5; 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5; 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5; 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5; 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5; 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5; 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
    return g, d
end

# Crea un conjunto de columnas tales que (P^K_0) es factible.
function crearColumnasIniciales()
    M1 = [[1 20], [3 2], [5 4], [7 6], [9 8], [11 10], [13 12], [15 14], [17 16], [19 18]]
    M2 = [[1 2], [3 4], [5 6], [7 8], [9 10], [11 12], [13 14], [15 16], [17 18], [19 20]]
    K_0 = []
    push!(K_0,M1)
    push!(K_0,M2)
    return K_0
end

crearColumnasIniciales (generic function with 1 method)

# 3. Problemas

Escriba sus respuestas aquí, tanto los códigos en Julia como respuestas a preguntas en Markdown.

## Problema 1
Implemente la función *resolverProblemaMaestro(m,A,g,d,K)* que crea un modelo para $(\mathcal{P}^{k})$, lo resuelva y retorne el valor objetivo, las variables primales y duales en el óptimo. Es importante que $K$ sea un parámetro para que permita ir agregando columnas.

*Indicación:* Puede apoyarse libremente en el siguiente template.

**Solución:**  El método es (usando el template entregado en el enunciado):

In [51]:
function resolverProblemaMaestro(m,A,g,d,K)
    maestro = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV), "Presolve"=>0, "Cuts"=>0, "Heuristics"=>0, "Threads"=>1))
    set_silent(maestro)
    
    n = size(K,1) #columnas
    A_k = zeros(m, n)
    for i in 1:n
        A_k[:,i]=A[:,K[i]]
    end    
    
    # Variables
    @variable(maestro, x[1:n] >= 0)
    
    # Funcion Objetivo
    @objective(maestro, Max, g' * A_k * x)
    
    # Restricciones 
    @constraint(maestro, primeraRestriccion, A_k * x .<= d)
    @constraint(maestro, segundaRestriccion, sum(x[j] for j in 1:n) == 1)
    
    optimize!(maestro)
    
    optimo = (termination_status(maestro) == MOI.OPTIMAL)
    if optimo
        valor_objetivo = objective_value(maestro)
        x = value.(maestro[:x])
        y = dual.(maestro[:primeraRestriccion])
        theta = dual.(maestro[:segundaRestriccion])

    else
        valor_objetivo, x, y, theta = "No hay optimo", 0, 0, 0
    end
    
    return valor_objetivo, x, y, theta
end

resolverProblemaMaestro (generic function with 1 method)

## Problema 2
Presente ordenadamente sus resultados.

In [52]:
for j in 1:11
    println("INSTANCIA "* string(j))
    A, g, d = crearInstancias(j)
    i = size(A,1)
    K = []
    for m in 1:i
        if m%2 == 0
            push!(K,m)
        end
    end
    #print(K)
    
    valor_objetivo, x, y, theta = resolverProblemaMaestro(i,A,g,d,K)
    if valor_objetivo == "No hay optimo"
        println(valor_objetivo)
    else    
        println("Valor objetivo: ",valor_objetivo)
        println("Variable primal:")
        println("x = ",x)
        println("Variables duales:")
        println("y =", y)
        println("theta = ", theta)
    end
    
end

INSTANCIA 1
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Heuristics to value 0
Set parameter Threads to value 1
No hay optimo
INSTANCIA 2
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Heuristics to value 0
Set parameter Threads to value 1
No hay optimo
INSTANCIA 3
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Heuristics to value 0
Set parameter Threads to value 1
No hay optimo
INSTANCIA 4
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Heuristics to value 0
Set parameter Threads to value 1
No hay optimo
INSTANCIA 5
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Heuristics to value 0
Set parameter Threads to value 1
Valor objetivo: 1.0246913580246912
Variable primal:
x = [0.8888888888888888, 0.1111111111111111, 0.0, 0.0]
Variables duales:
y =[-0.0, -0.0, -0.0, -0.0, -0.7777777777777777, -0.0, -0.0, -0.0, -0.0]
theta = -1.1111111111

In [53]:
A, g, d = crearInstancias(5)
i = size(A,1)
K = []
for m in 1:i
    if m%2 == 0
        push!(K,m)
    end
end
resolverProblemaMaestro(i,A,g,d,K)

Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Heuristics to value 0
Set parameter Threads to value 1


(1.0246913580246912, [0.8888888888888888, 0.1111111111111111, 0.0, 0.0], [-0.0, -0.0, -0.0, -0.0, -0.7777777777777777, -0.0, -0.0, -0.0, -0.0], -1.1111111111111112)

## P3
Escriba y justifique su modelo lineal entero.

**Solución:** Sean $X$ e $Y$ dos conjuntos de cardinal $N$, luego podemos identificar cada emparejamiento maximal $M(X,Y)$ con una permutación $\sigma:\{1,\dots,N\}\to\{1,\dots,N\}$. Luego, sabemos que la cantidad de permutaciones posibles es $N!$.

Luego sea $A$ una matriz tal que cada columna representa una configuración de emparejamiento posible sobre los conjuntos $X$ e $Y$, en cuyo caso, notemos que $A$ tiene $N^{2}$ filas pues es la cantidad de aristas que tiene un grafo bipartito completo, y tiene $N!$ columnas, pues cada columna representa un posible emparejamiento.

Sea $(\overline{y},\overline{\theta})$ la solución óptima de $(\mathcal{D}^{K})$. Definimos $c := \overline{y} - g$, luego estudiamos el problema 
\begin{equation}
    (\mathcal{E})\qquad \min_{j\in[n]} (A^{t}(\overline{y} - g))j    
\end{equation}
donde
\begin{equation}
    (A^{t}c)_{j} = \sum_{i=1}^{N^{2}} A_{ij} c_{i}, 
\end{equation}
entonces podemos escribir $A_{\cdot j}$ es la indicatriz del emparejamiento $j$-ésimo y buscamos minimizar el costo de dicho emparejamiento dado por la *función de costos* $c$ definida antes.

Planteamos el problema de optimización a resolver como:
\begin{eqnarray*}
    (\mathcal{E}') \left\{
    \begin{array}{rll}
    \min        & c^{t}x                                        &                         \\
    \text{s.a.} & \displaystyle\sum_{i=jN+1}^{(j+1)N} x_{i} = 1 & (\forall j=0,\dots,N-1) \\
                & \displaystyle\sum_{j=0}^{N-1} x_{i+jN} = 1    & (\forall i=1,\dots,N)   \\
                & x_{i+(j-1)N} \leq 0,                          & (\forall i,j=1,\dots,N : ij \not \in E) \\
                & x_{i}\in\{0,1\}                               & (\forall i=1,\dots,N^2) \\
    \end{array}\right.
\end{eqnarray*}



## P4
Puede apoyarse libremente en el siguiente template.

In [79]:
function resolverProblemaPricing(V,E,g,y)
    pricing = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV), "Presolve"=>0, "Cuts"=>0, "Heuristics"=>0, "Threads"=>1))
    set_silent(pricing)
    
#     @variable(pricing, variables con sus dominios)
#     @objective(pricing, Min, expresion_de_la_funcion_objetivo)
#     @constraints(pricing, begin
#         primeraRestriccion ...
#         segundaRestriccion ...
#     end)
    
    m=size(E)[1]
    
    @variable(pricing,x[1:V^2],Bin)
    for j=0:(V-1)
        @constraint(pricing,sum(x[i]       for i= (j*V+1):(j*V+V)) == 1)
        @constraint(pricing,sum(x[j+1+i*V] for i= 0:(V-1))         == 1)
    end
    
    y2 = zeros(V^2)
    g2 = zeros(V^2)
    
    #=
    for i=1:V
        for j=1:V
            t=true
            for k=1:m
                if E[k] == [i,j]
                    t=false
                end
            end
            if t
                @constraint(pricing,x[j + (i-1)*V] == 0)
            end
        end
    end
    =#
    
    for k=1:m
        i,j=E[k]
        y2[j + (i-1)*V] = y[k]
        g2[j + (i-1)*V] = g[k]
    end
   
    
    
    @objective(pricing, Min, (y2-g2)' * x)
        
    
    optimize!(pricing)
    
    if termination_status(pricing) != MOI.OPTIMAL
        @warn("No hay optimo")
    end
    
    valor_objetivo = objective_value(pricing)
    equis = value.(pricing[:x])
    
    
    # Dada esta solución, como la función de costos es igual a 0 en las coordenadas de x que no son aristas del grafo original,
    # Se pueden cambiar a 0 si es necesario y la solución sigue siendo óptima para el problema original.
    for i=1:V
        for j=1:V
            t=true
            for k=1:m
                if E[k] == [i,j]
                    t=false
                end
            end
            if t
               equis[j + (i-1)*V] = 0
            end
        end
    end
    
    
    
    return valor_objetivo, equis
end

resolverProblemaPricing (generic function with 1 method)

## P5
Presente ordenadamente la solución.



In [80]:
V,m,E = crearGrafoBipartito()
y = crearTestY()
g = zeros(m)

resolverProblemaPricing(V,E,g,y)


Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Heuristics to value 0
Set parameter Threads to value 1


(0.0, [0.0, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, -0.0, 0.0, -0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 4.0)

## P6
Puede apoyarse libremente en el siguiente template.

In [56]:
function resolverProblemaMaestro2(V,g,d,K)
    maestro = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV), "Presolve"=>0, "Cuts"=>0, "Heuristics"=>0, "Threads"=>1))
    set_silent(maestro)
    
    n,m = size(K) #columnas
    
    A_k=zeros(V^2,factorial(V))
    
    for i=1:n
        for j=1:m
            a,b=K[i][j]
            A[i,(a-1)*V + b] = 1
        end
    end
    
    # Variables
    @variable(maestro, x[1:n] >= 0)
    
    # Funcion Objetivo
    @objective(maestro, Max, g' * A_k * x)
    
    # Restricciones 
    @constraint(maestro, primeraRestriccion, A_k * x .<= d)
    @constraint(maestro, segundaRestriccion, sum(x[j] for j in 1:n) == 1)
    
    optimize!(maestro)
    
    optimo = (termination_status(maestro) == MOI.OPTIMAL)
    if optimo
        valor_objetivo = objective_value(maestro)
        x = value.(maestro[:x])
        y = dual.(maestro[:primeraRestriccion])
        theta = dual.(maestro[:segundaRestriccion])

    else
        valor_objetivo, x, y, theta = "No hay optimo", 0, 0, 0
    end
    
    return valor_objetivo, x, y, theta
end

resolverProblemaMaestro2 (generic function with 1 method)

In [6]:
function resolverGeneracionColumnas(m,E,g,d,K_0)
    K = K_0
    
    maxcols = 1000 # damos una cota en el número de columnas a usar.

    while length(K) <= maxcols
        # Paso 1. Resolver problema maestro (P^K)
        valor_objetivo, x, y, theta = resolverProblemaMaestro2(m,g,d,K)
        # Paso 2. Resolver Pricing (E)
        valoreps, z = resolverProblemaPricing(m,E,g,y)
        # Paso 3. Revisar si óptimo global o si se necesita una nueva columna
        if optimo
            println("Óptimo")
            return K, variable_primal, valor_objetivo
        else
            push!(K, nueva_columna)
        end
    end
    
    # Se alcanzó el número de iteraciones
    println("Subóptimo")
    return K, variable_primal, valor_objetivo
end

resolverGeneracionColumnas (generic function with 1 method)

## P7
Presente ordenadamente la solución.